# Dash

https://dash.plot.ly/getting-started

Dash is a Python framework for building web apps. It's made by Plotly, so no surprise that it works well for building Dashboards-like apps with analytics and visualization at their core.

With Dash we can make *real* full-service apps for users to do their own exploration and analysis. Here's a good one: https://dash-gallery.plotly.host/dash-yield-curve/

### Same old data

In [14]:
import pandas as pd
temperatures = pd.read_csv("../data/global_temperatures/GlobalLandTemperaturesByCountry.csv", parse_dates=['dt'])
continents = pd.read_csv("../data/continents.csv")
countries = pd.read_csv("../data/countries.csv")
countries.drop(columns=['code'])
temperatures = temperatures.merge(continents).merge(countries, left_on="Country", right_on="country")

temperatures['year'] = temperatures.dt.dt.year

yearly_change = temperatures[(temperatures.year==1963) | (temperatures.year==2013)].groupby(["Country","year"], as_index=False).AverageTemperature.mean()
yearly_change['AverageTemperatureChange'] = yearly_change.groupby(["Country"], as_index=False).AverageTemperature.transform("diff")
yearly_change.dropna(inplace=True)
temperature_slice=yearly_change.merge(temperatures[["Country","Continent","Code","lon","lat"]].drop_duplicates())

# The Layout

The layout is the *wireframe* or the *structure* that the app is built on. It is at its core `HTML`, the language of the (backbone of the?) web. HTML elements you may be familiar with are accessible through Python objects. This may seem strange at first, but it makes it so that we can build all of this in one place and one context.

### dash_html_components

The dash_html_components library has a component for every HTML tag. The `html.H1(children='Hello Dash')` component generates a `<h1>Hello Dash</h1>` HTML element in your application.

### dash_core_components

Not all components are pure HTML. The `dash_core_components` describe higher-level components that are interactive and are generated with JavaScript, HTML, and CSS through the React.js library.

In [4]:
from werkzeug.serving import run_simple as _run_simple
def debug(app, port=8000, host="0.0.0.0"):
    """
    Run a dash app with dash dev tools from within Jupyter Notebooks
    """
    if type(app) == dash.dash.Dash:
        # right, this is a Dash app?
        app.enable_dev_tools()
        app = app.server
    _run_simple(host, port, app)


In [8]:
import dash
import dash_core_components as dcc
import dash_html_components as html

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(children=[
    html.H1(children='Hello Dash'),

    html.Div(children='''
        Dash: A web application framework for Python.
    '''),

    dcc.Graph(
        id='example-graph',
        figure={
            'data': [
                {'x': [1, 2, 3], 'y': [4, 1, 2], 'type': 'bar', 'name': 'SF'},
                {'x': [1, 2, 3], 'y': [2, 4, 5], 'type': 'bar', 'name': u'Montréal'},
            ],
            'layout': {
                'title': 'Dash Data Visualization'
            }
        }
    )
])

debug(app)

# EXERCISE

- Replace the figure one that we like

In [23]:
import plotly.express as px

figure = px.scatter(
    temperature_slice,
    x="AverageTemperature",
    y="AverageTemperatureChange",
    color="Continent",
    marginal_y="violin",
    marginal_x="histogram",
    hover_data=["Country"]
)

app.layout = html.Div(children=[
    html.H1(children='Hello Dash'),

    html.Div(children='''
        Dash: A web application framework for Python.
    '''),

    dcc.Graph(
        id='example-graph',
        figure=figure
    )
])

debug(app)

In [ ]:
# Your Solution

# Callbacks

This is a term taken primarily from JavaScript, which is the *real* language of the web. 

## EXERCISE

- Add a dropdown menu to choose the viz type
- Tie it to a callback to refresh the plot

# Click Events

## EXERCISE

- Capture click event on plot and pull up the related Countries full history